In [2]:
import psycopg2
import pandas as pd
import vaex
import requests
import pandas as pd
from joblib import Parallel, delayed
import time
import networkx as nx
from bs4 import BeautifulSoup
import pycountry
import re
from create_df_from_sql import *
import os
import sys

In [ ]:
base_dir = '/home/reutme/Big_data/final_project'
code_dir = os.path.join(base_dir, 'code')
sys.path.append(code_dir)
import const
import params

## OpenAlex API

In [3]:
# Base URL for OpenAlex API
BASE_URL = "https://api.openalex.org"

In [4]:
def extract_author_id(author_url):
    author_id = author_url.split("/")[-1]
    return author_id

In [5]:
def fetch_author_info(author_id):
    """Fetch detailed author information by author ID."""
    url = f"{BASE_URL}/authors/{author_id}"
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    else:
        raise Exception(f"Failed to fetch author info: {response.status_code}")


# Function to fetch author information with retry mechanism
def fetch_author_info_delay(author_id, retries=3, delay=2):
    url = f"https://api.openalex.org/authors/{author_id}"
    for attempt in range(retries):
        response = requests.get(url)
        if response.status_code == 200:
            return response.json()
        elif response.status_code == 429:
            print("Rate limit exceeded. Retrying...")
            time.sleep(delay)  # Wait before retrying
        else:
            print(f"Failed to fetch data for author ID {author_id}: {response.status_code}")
            return None
    print(f"Failed to fetch data for author ID {author_id} after {retries} retries.")
    return None

def fetch_author_works(author_id):
    """Fetch works associated with an author by their ID."""
    url = f"{BASE_URL}/works?filter=author.id:{author_id}"
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()["results"]
    else:
        raise Exception(f"Failed to fetch works: {response.status_code}")

def evaluate_impact(author_data):
    """Evaluate research impact using h_index and i10_index."""
    impact_data = author_data['summary_stats']
    h_index = impact_data.get("h_index", "Not available")
    i10_index = impact_data.get("i10_index", "Not available")
    return {"h_index": h_index, "i10_index": i10_index}

def analyze_topics(author_data):
    """Analyze main areas of research interest."""
    topics = author_data.get("topics", [])
    return [topic["display_name"] for topic in topics]

def explore_affiliations(author_data):
    """Explore institutional affiliations."""
    affiliations = author_data.get("affiliations", [])
    institutions = [
        {"name": inst['institution'].get("display_name", "Unknown"), "country": inst['institution'].get("country_code", "Unknown"), "years": inst['years']}
        for inst in affiliations
    ]
    return institutions


In [6]:
def process_author(author_id):
    author_data = fetch_author_info_delay(author_id)
    if not author_data:
        return None

    author_name = author_data.get('display_name', "Unknown")    
    affiliations = explore_affiliations(author_data)
    
    rows = []
    for affiliation in affiliations:
        institution_name = affiliation['name']
        country = affiliation['country']
        years = affiliation['years']
        
        # Create a row for each year
        for year in years:
            rows.append({
                "author_id": author_id,
                "author_name": author_name,
                "institution_name": institution_name,
                "country": country,
                "year": year
            })
    
    return pd.DataFrame(rows)


In [7]:

url = "https://api.openalex.org/authors"
params = {
    "page": 1, 
    "per_page": 200,
}

response = requests.get(url, params=params)
if response.status_code == 200:
    authors = response.json().get("results", [])
    author_ids = [extract_author_id(author["id"]) for author in authors]
# for author_id in author_ids:
#     author_dfs = process_author(author_id)
#     print(author_dfs)
author_dfs = Parallel(n_jobs=2)(delayed(process_author)(author_id) for author_id in author_ids)
final_df = pd.concat(author_dfs, ignore_index=True)


In [8]:
final_df.head()

,author_id,author_name,institution_name,country,year
0,A5010062957,A Boyle,University of Michigan–Ann Arbor,US,2022
1,A5010062957,A Boyle,University of Michigan–Ann Arbor,US,2021
2,A5010062957,A Boyle,Macquarie University,AU,2022
3,A5022654839,Terry Law,Environmental Molecular Sciences Laboratory,US,2023
4,A5022654839,Terry Law,Environmental Molecular Sciences Laboratory,US,2022


In [11]:
final_df = final_df.dropna()

In [13]:
unique_countries_list = final_df['country'].unique().tolist()
unique_countries_list.sort()
print(unique_countries_list)

['AE', 'AR', 'AT', 'AU', 'BD', 'BE', 'BG', 'BR', 'BY', 'CA', 'CH', 'CN', 'CZ', 'DE', 'DK', 'EG', 'ES', 'FR', 'GB', 'IE', 'IN', 'IT', 'JO', 'JP', 'KR', 'KZ', 'MO', 'MX', 'MY', 'NG', 'NL', 'NP', 'NZ', 'PL', 'QA', 'RE', 'RO', 'RS', 'RU', 'SA', 'SE', 'SG', 'SK', 'TH', 'TW', 'UA', 'US', 'UY', 'UZ', 'VN', 'ZA', 'ZW']


## OpenAlex Data Base

In [2]:
PASSWORD = const.PASSWORD
# Database connection parameters
db_params = const.db_params

### Explore the data base

In [4]:
try:
    # Establish a connection to the PostgreSQL database
    connection = psycopg2.connect(**db_params)    
# Create a cursor object to interact with the database
    cursor = connection.cursor()
    
    # Example query to test the connection
    cursor.execute("SELECT version();")
    db_version = cursor.fetchone()
    print(f"Connected to PostgreSQL database. Version: {db_version[0]}")

except Exception as e:
    print(f"An error occurred: {e}")
finally:
    # Close the connection (important to release resources)
    if 'connection' in locals() and connection:
        connection.close()
        print("Connection closed.")

Connected to PostgreSQL database. Version: PostgreSQL 12.2 on x86_64-pc-linux-gnu, compiled by gcc (GCC) 4.8.5 20150623 (Red Hat 4.8.5-39), 64-bit
Connection closed.


Extract from works_authorships the work id, author id and institution id

In [43]:
query = """
    SELECT * 
    FROM openalex.works_authorships as wa
    WHERE wa.work_id IS NOT NULL AND wa.author_id IS NOT NULL AND wa.institution_id IS NOT NULL
    LIMIT 10;
"""

In [44]:
df_connection_all = sql_query(db_params, query)
df_connection_all.head(5)

Total rows fetched: 10


#,work_id,author_position,author_id,institution_id,raw_affiliation_string
0,https://openalex.org/W4379231911,middle,https://openalex.org/A5027283708,https://openalex.org/I4210128919,"'Bourgogne Franche-Comté University, INSERM, Eta..."
1,https://openalex.org/W4379231911,middle,https://openalex.org/A5027283708,https://openalex.org/I154526488,"'Bourgogne Franche-Comté University, INSERM, Eta..."
2,https://openalex.org/W4379231911,middle,https://openalex.org/A5032052791,https://openalex.org/I4210118524,"'Bourgogne Franche-Comté University, INSERM, Eta..."
3,https://openalex.org/W4379231911,middle,https://openalex.org/A5032052791,https://openalex.org/I4210128919,"'Bourgogne Franche-Comté University, INSERM, Eta..."
4,https://openalex.org/W4379231911,middle,https://openalex.org/A5032052791,https://openalex.org/I154526488,"'Bourgogne Franche-Comté University, INSERM, Eta..."


Extract institution attributes based on institution id

In [45]:
query = """
    SELECT * 
    FROM openalex.institutions AS inst_table
    WHERE inst_table.id = 'https://openalex.org/I187531555';
"""
sql_query(db_params, query)

Total rows fetched: 1


#,id,ror,display_name,country_code,type,homepage_url,image_url,image_thumbnail_url,display_name_acronyms,display_name_alternatives,works_count,cited_by_count,works_api_url,updated_date
0,https://openalex.org/I187531555,https://ror.org/012a77v79,Lund University,SE,education,http://www.lunduniversity.lu.se/,'https://commons.wikimedia.org/w/index.php?title...,'https://commons.wikimedia.org/w/index.php?title...,"array([], dtype=float64)","array([], dtype=float64)",170085,5463912,'https://api.openalex.org/works?filter=instituti...,"'datetime.datetime(2023, 10, 19, 11, 29, 46, 702..."


Extract author attributes based on author id

In [ ]:
query = """
    SELECT * 
    FROM openalex.authors AS auth
    WHERE auth.id = 'https://openalex.org/A5013167854';
"""

In [46]:
sql_query(db_params, query)

Total rows fetched: 1


#,id,ror,display_name,country_code,type,homepage_url,image_url,image_thumbnail_url,display_name_acronyms,display_name_alternatives,works_count,cited_by_count,works_api_url,updated_date
0,https://openalex.org/I187531555,https://ror.org/012a77v79,Lund University,SE,education,http://www.lunduniversity.lu.se/,'https://commons.wikimedia.org/w/index.php?title...,'https://commons.wikimedia.org/w/index.php?title...,"array([], dtype=float64)","array([], dtype=float64)",170085,5463912,'https://api.openalex.org/works?filter=instituti...,"'datetime.datetime(2023, 10, 19, 11, 29, 46, 702..."


Extract work attributes based on work id

In [3]:
query = """
    SELECT * 
    FROM openalex.works AS work
    WHERE work.id = 'https://openalex.org/W3198663116';
"""

In [4]:
df = sql_query(db_params, query)

Connection to the database established.
Cursor created successfully.
Query executed successfully.
No more rows to fetch. Exiting.
Total shape fetched: (1, 13)
Database connection closed.


In [10]:
query_w = """SELECT id AS work_id, publication_year, publication_date, cited_by_count 
            FROM openalex.works as w 
            WHERE w.publication_year > 2010 LIMIT 200;"""
sql_query(db_params, query_w)

Total rows fetched: 200


#,work_id,publication_year,publication_date,cited_by_count
0,https://openalex.org/W1588569265,2011,2011-02-04,0
1,https://openalex.org/W1588573310,2021,2021-08-26,0
2,https://openalex.org/W1588573632,2013,2013-01-01,0
3,https://openalex.org/W1588573947,2014,2014-01-01,0
4,https://openalex.org/W1588574714,2014,2014-10-01,0
...,...,...,...,...
195,https://openalex.org/W1588833533,2012,2012-01-01,0
196,https://openalex.org/W1588834031,2012,2012-08-27,0
197,https://openalex.org/W1588834755,2012,2012-11-15,0
198,https://openalex.org/W1588835316,2014,2014-01-01,0


### Extract all the data using sql query

In [10]:
query_all = """
    SELECT 
        wa.work_id,
        wa.author_id,
        wa.institution_id,
        w.publication_year,
        w.publication_date,
        w.cited_by_count,
        a.display_name AS author_name,
        i.display_name AS institution_name,
        i.country_code
    FROM openalex.works_authorships AS wa
    JOIN openalex.works AS w ON wa.work_id = w.id
    JOIN openalex.authors AS a ON wa.author_id = a.id
    JOIN openalex.institutions AS i ON wa.institution_id = i.id
    WHERE wa.work_id IS NOT NULL AND wa.author_id IS NOT NULL AND wa.institution_id IS NOT NULL
    LIMIT 1000;
"""

In [51]:
sql_query(db_params, query_all).head(5)

Total rows fetched: 1000


#,work_id,author_id,institution_id,publication_year,publication_date,cited_by_count,author_name,institution_name,country_code
0,https://openalex.org/W1552429227,https://openalex.org/A5026835600,https://openalex.org/I177055848,2013,2013-04-17,0,Achmad Wicaksono,University of Brawijaya,ID
1,https://openalex.org/W1552429227,https://openalex.org/A5070253412,https://openalex.org/I177055848,2013,2013-04-17,0,Harnen Sulistio,University of Brawijaya,ID
2,https://openalex.org/W1552429227,https://openalex.org/A5057585820,https://openalex.org/I177055848,2013,2013-04-17,0,Ludfi Djakfar,University of Brawijaya,ID
3,https://openalex.org/W1552429227,https://openalex.org/A5056862134,https://openalex.org/I177055848,2013,2013-04-17,0,Hendi Bowoputro,University of Brawijaya,ID
4,https://openalex.org/W1552431148,https://openalex.org/A5091060974,https://openalex.org/I150468666,2014,2014-11-25,0,Pratik Pimple,Emory University,US


### Extract each table using sql query and vaex

In [35]:
query_wa = """
    SELECT  work_id, author_id, institution_id 
    FROM openalex.works_authorships as wa
    WHERE wa.work_id IS NOT NULL AND wa.author_id IS NOT NULL AND wa.institution_id IS NOT NULL;
"""
query_w = """SELECT id AS work_id, publication_year, publication_date, cited_by_count 
            FROM openalex.works as w 
            WHERE w.publication_year > 1900;"""

query_a = "SELECT id AS author_id, display_name AS author_name FROM openalex.authors;"
query_i = "SELECT id AS institution_id, display_name AS institution_name, country_code FROM openalex.institutions;"

In [36]:
df_w = sql_query(db_params, query_w, 'works_df.csv')
df_wa = sql_query(db_params, query_wa, 'work_inst_auth_df.csv')
df_a = sql_query(db_params, query_a, 'author_df.csv')
df_i = sql_query(db_params, query_i, 'inst_df.csv')

Total rows fetched: 1000000
Data saved to output.csv successfully!


### Merge csv to one dataframe

In [4]:
main_path = os.path.join(const.data_base_dir,"data_little_amount")

In [5]:
path_w = f'{main_path}/works_df.csv'
path_wa = f'{main_path}/work_inst_auth_df.csv'
path_a = f'{main_path}/author_df.csv'
path_i = f'{main_path}/inst_df.csv'

In [6]:
df_w = vaex.read_csv(path_w, convert=True, progress=True)
df_wa = vaex.read_csv(path_wa, convert=True, progress=True)
df_a = vaex.read_csv(path_a, convert=True, progress=True)
df_i = vaex.read_csv(path_i, convert=True, progress=True)


Converting csv to chunk files
Saved chunk #0 to /home/reutme/Big_data/final_project/data/data_little_amount/works_df.csv_chunk_0.hdf5
Converting csv to chunk files
Saved chunk #0 to /home/reutme/Big_data/final_project/data/data_little_amount/work_inst_auth_df.csv_chunk_0.hdf5
Converting csv to chunk files
Saved chunk #0 to /home/reutme/Big_data/final_project/data/data_little_amount/author_df.csv_chunk_0.hdf5
Converting csv to chunk files
Saved chunk #0 to /home/reutme/Big_data/final_project/data/data_little_amount/inst_df.csv_chunk_0.hdf5


In [7]:
df_merged = df_wa.join(df_w, on="work_id", how="left")

# Join the result with df_a on "author_id"
df_merged = df_merged.join(df_a, on="author_id", how="left")

# Join the result with df_i on "institution_id"
df_merged = df_merged.join(df_i, on="institution_id", how="left")

# Step 3: Display the merged DataFrame
print(df_merged)

#    work_id                          author_id                         institution_id                    publication_year    publication_date    cited_by_count    author_name    institution_name    country_code
0    https://openalex.org/W399541487  https://openalex.org/A5018056130  https://openalex.org/I121820613   --                  --                  --                --             --                  --
1    https://openalex.org/W399542285  https://openalex.org/A5069860388  https://openalex.org/I197251160   --                  --                  --                --             --                  --
2    https://openalex.org/W399542285  https://openalex.org/A5069860388  https://openalex.org/I4210141777  --                  --                  --                --             --                  --
3    https://openalex.org/W399542285  https://openalex.org/A5069860388  https://openalex.org/I4210090701  --                  --                  --                --             -- 

In [10]:
print(f"merge data frame 5 first rows: /n{df_merged.head(5)}")


merge data frame 5 first rows: /n  #  work_id                          author_id                         institution_id                    publication_year    publication_date    cited_by_count    author_name    institution_name    country_code
  0  https://openalex.org/W399541487  https://openalex.org/A5018056130  https://openalex.org/I121820613   --                  --                  --                --             --                  --
  1  https://openalex.org/W399542285  https://openalex.org/A5069860388  https://openalex.org/I197251160   --                  --                  --                --             --                  --
  2  https://openalex.org/W399542285  https://openalex.org/A5069860388  https://openalex.org/I4210141777  --                  --                  --                --             --                  --
  3  https://openalex.org/W399542285  https://openalex.org/A5069860388  https://openalex.org/I4210090701  --                  --                  -- 

## Read full data frame

In [2]:
# Path to the Parquet file
file_path = os.path.join(const.data_base_dir, "OpenAlex_merged_data.parquet")

# Read the Parquet file into a Vaex dataframe
df = vaex.open(file_path)

df.head(5)


#,work_id,author_id,institution_id,publication_year,publication_date,cited_by_count,author_name,institution_name,country_code
0,https://openalex.org/W3126688306,https://openalex.org/A5021130645,https://openalex.org/I2799504677,2009,2009-12-31,0,Carbonne Pierre,"""Institut de l'Audiovisuel et des Télécommunicat...",FR
1,https://openalex.org/W3126688306,https://openalex.org/A5030528300,https://openalex.org/I2799504677,2009,2009-12-31,0,Thomas Hain,"""Institut de l'Audiovisuel et des Télécommunicat...",FR
2,https://openalex.org/W3126688306,https://openalex.org/A5038081513,https://openalex.org/I190778170,2009,2009-12-31,0,Gary Clemo,Ofcom,GB
3,https://openalex.org/W3126688432,https://openalex.org/A5082259378,https://openalex.org/I130701412,2020,2020-12-23,0,Віра Борщовецька,Chernivtsi National University,UA
4,https://openalex.org/W3126688432,https://openalex.org/A5045080228,https://openalex.org/I130701412,2020,2020-12-23,0,Марія Рубанець,Chernivtsi National University,UA


In [4]:
filtered_df = df[df['publication_year'] == 2022]

print(filtered_df.head(5))

  #  work_id                           author_id                         institution_id                      publication_year  publication_date      cited_by_count  author_name                  institution_name                                     country_code
  0  https://openalex.org/W3126695662  https://openalex.org/A5014855960  https://openalex.org/I4210160450                2022  2022-01-01                         0  Aurélien Manchon             'Centre Interdisciplinaire de Nanoscience de Mar...  FR
  1  https://openalex.org/W3126749003  https://openalex.org/A5044210345  https://openalex.org/I2800562746                2022  2022-03-01                         0  E. Carlos Rodríguez-Merchán  Hospital Universitario La Paz                        ES
  2  https://openalex.org/W3126853602  https://openalex.org/A5044210345  https://openalex.org/I2800562746                2022  2022-07-01                         0  E. Carlos Rodríguez-Merchán  Hospital Universitario La Paz                 

In [5]:
# Get the shape of the DataFrame
num_rows = len(df)  # Number of rows
num_columns = len(df.get_column_names())  # Number of columns

# Print the shape
print(f"Number of rows: {num_rows}")
print(f"Number of columns: {num_columns}")


Number of rows: 365328026
Number of columns: 9


In [5]:
df['country_code'].value_counts()

missing       24901
US         85285246
CN         51488500
FR         22687147
JP         19769819
             ...   
TV               68
CK                9
BQ                3
VI                3
EH                1
Length: 226, dtype: int64

In [6]:
# Get the maximum value of the column
max_value = df['publication_year'].max()

# Print the maximum value
print(f"The maximum value in the column '{'publication_year'}' is: {max_value}")

The maximum value in the column 'publication_year' is: 2025
